In [1]:
import os, json, requests
from jellyfish import levenshtein_distance, hamming_distance, jaro_distance, jaro_winkler

In [2]:
key    = 'EE44CC2E8EDE2A1288B72CA1D41D0462'
url_fl = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={0}&steamid={1}'
url_sm = 'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={0}&steamids={1}'
url_og = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={0}&steamid={1}'
url_ac = 'http://api.steampowered.com/ISteamUserStats/GetPlayerAchievements/v0001/?appid={0}&key={1}&steamid={2}'
url_bn = 'http://api.steampowered.com/ISteamUser/GetPlayerBans/v1/?key={0}&steamids={1}'
url_lo = 'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={0}&steamids={1}'

___

In [3]:
def LenFriendList(sid):
    url  = url_fl.format(key, sid)
    resp = requests.get(url)
    data = json.loads(resp.content)
    
    try:
        return len(data['friendslist']['friends'])
    except:
        return -1

In [4]:
print('Reading ids.txt')
with open('Data/ids.txt', 'r') as f:
    ids = eval(f.read())

print('Reading friends.txt')
with open('Data/friends.txt', 'r') as f:
    lfls = eval(f.read())

done = len(lfls)
print('Getting friend lists.')
try:
    for idx, sid in enumerate(ids):
        if idx >= done:
            lfl = LenFriendList(sid)
            lfls.append(lfl)
            print(idx, lfl)
    else:
        print('Done getting friends!')
finally:
    print('Writing to friends.txt')
    with open('Data/friends.txt', 'w') as f:
        f.write(repr(lfls))

Reading ids.txt
Reading friends.txt
Getting friend lists.
Done getting friends!
Writing to friends.txt


___

In [5]:
def GetNames(sid):
    rn, pn = u'', u''
    
    try:
        url  = url_sm.format(key, sid)
        resp = requests.get(url)
        data = json.loads(resp.content)

        player = data['response']['players'][0]
        pn = player['personaname']
        rn = player['realname']
    finally:
        return rn, pn

In [6]:
def GetDistances(rn, pn):
    dl, dh, dj, jw = -1, -1, -1, -1
    
    try:
        dl = levenshtein_distance(rn, pn)
        dh = hamming_distance(rn, pn)
        dj = jaro_distance(rn, pn)
        jw = jaro_winkler(rn, pn)
    finally:
        return dl, dh, dj, jw

In [7]:
print('Reading ids.txt')
with open('Data/ids.txt', 'r') as f:
    ids = eval(f.read())

print('Reading names.txt')
with open('Data/names.txt', 'r') as f:
    names = eval(f.read())

done  = len(names)
print('Getting names.')
try:
    for idx, sid in enumerate(ids):
        if idx >= done:
            rn, pn = GetNames(sid)
            dl, dh, dj, jw = GetDistances(rn, pn)
            names.append((rn, pn, dl, dh, dj, jw))
            print(idx, rn, pn)
    else:
        print('Done getting names!')
finally:
    print('Writing to names.txt')
    with open('Data/names.txt', 'w') as f:
        f.write(repr(names))

Reading ids.txt
Reading names.txt
Getting names.
Done getting names!
Writing to names.txt


___

In [8]:
def GetGames(sid):
    try:
        url  = url_og.format(key, sid)
        resp = requests.get(url)
        data = json.loads(resp.content)
        
        games = []
        for game in data['response']['games']:
            games.append((game['appid'], game['playtime_forever']))
        games.sort(key=lambda x: x[0])
    finally:
        return games

In [9]:
print('Reading ids.txt')
with open('Data/ids.txt', 'r') as f:
    ids = eval(f.read())

print('Reading games.txt')
with open('Data/games.txt', 'r') as f:
    games = eval(f.read())

done  = len(games)
print('Getting games.')
try:
    for idx, sid in enumerate(ids):
        if idx >= done:
            og = GetGames(sid)
            games.append(og)
            print(idx, len(og))
    else:
        print('Done getting games!')
finally:
    print('Writing to games.txt')
    with open('Data/games.txt', 'w') as f:
        f.write(repr(games))

Reading ids.txt
Reading games.txt
Getting games.
Done getting games!
Writing to games.txt


___

In [10]:
def GetAchievements(appid, sid):
    try:
        url  = url_ac.format(appid, key, sid)
        resp = requests.get(url)
        data = json.loads(resp.content)
        
        return data['playerstats']['achievements']
    except:
        return -1

In [11]:
print('Reading ids.txt')
with open('Data/ids.txt', 'r') as f:
    ids = eval(f.read())

print('Reading games.txt')
with open('Data/games.txt', 'r') as f:
    gamess = eval(f.read())

print('Reading achievements.txt')
with open('Data/achievements.txt', 'r') as f:
    achieves = eval(f.read())

done = len(achieves)
print('Getting achievements.')
try:
    for idx, (sid, games) in enumerate(zip(ids, gamess)):
        if idx >= done:
            amount, gotten, total = (len(games), 0, 0)
            print(idx, amount)
            
            for game in games:
                achievements = GetAchievements(game[0], sid)
                if achievements != -1:
                    for a in achievements:
                        gotten += a['achieved']
                    total += len(achievements)

            achieves.append((amount, gotten, total))
    else:
        print('Done getting achievements!')
finally:
    print('Writing to achievements.txt')
    with open('Data/achievements.txt', 'w') as f:
        f.write(repr(achieves))

Reading ids.txt
Reading games.txt
Reading achievements.txt
Getting achievements.
Done getting achievements!
Writing to achievements.txt


___

In [12]:
def GetBans(sid):
    try:
        url  = url_bn.format(key, sid)
        resp = requests.get(url)
        data = json.loads(response.content)
        
        player = json_obj['players'][0]
        return player['NumberOfVACBans'], player['DaysSinceLastBan'], player['NumberOfGameBans']
    except:
        return -1, -1, -1

In [13]:
print('Reading ids.txt')
with open('Data/ids.txt', 'r') as f:
    ids = eval(f.read())

print('Reading bans.txt')
with open('Data/bans.txt', 'r') as f:
    bans = eval(f.read())

done = len(bans)
print('Getting bans.')
try:
    for idx, sid in enumerate(ids):
        if idx >= done:
            tuple_bans = GetBans(sid)
            bans.append(tuple_bans)
            print(idx, tuple_bans[1])
    else:
        print('Done getting bans!')
finally:
    print('Writing to bans.txt')
    with open('Data/bans.txt', 'w') as f:
        f.write(repr(bans))

Reading ids.txt
Reading bans.txt
Getting bans.
Done getting bans!
Writing to bans.txt


___

In [14]:
def GetLastlogoff(sid):
    url  = url_lo.format(key, sid)
    resp = requests.get(url)
    data = json.loads(resp.content)
        
    return data['response']['players'][0]['lastlogoff']

In [15]:
print('Reading ids.txt')
with open('Data/ids.txt', 'r') as f:
    ids = eval(f.read())

print('Reading lastlogoff.txt')
with open('Data/lastlogoff.txt', 'r') as f:
    lastlogoff = eval(f.read())

done = len(lastlogoff)
print('Getting lastlogoff.')
try:
    for idx, sid in enumerate(ids):
        if idx >= done:
            llo = GetLastlogoff(sid)
            lastlogoff.append(llo)
            print(idx, llo)
    else:
        print('Done getting lastlogoff!')
finally:
    print('Writing to lastlogoff.txt')
    with open('Data/lastlogoff.txt', 'w') as f:
        f.write(repr(lastlogoff))

Reading ids.txt
Reading lastlogoff.txt
Getting lastlogoff.
Done getting lastlogoff!
Writing to lastlogoff.txt
